In [1]:
# importing libraries
import pandas as pd # data science essentials
import matplotlib.pyplot as plt # essential graphical output
import seaborn as sns # enhanced graphical output
import numpy as np # mathematical essentials
import statsmodels.formula.api as smf # regression modeling
from sklearn.model_selection import train_test_split # train/test split
from sklearn.linear_model import LinearRegression # linear regression (scikit-learn)
import sklearn.linear_model # linear models

# specifying file name
file = './birthweight_low.xlsx'

# reading the file into Python
bwdf_low = pd.read_excel(io = file,
                         sheet_name = 0,
                         header = 0)
bwdf_low.head()

,mage,meduc,monpre,npvis,fage,feduc,omaps,fmaps,cigs,drink,male,mwhte,mblck,moth,fwhte,fblck,foth,bwght
0,69,NaN,5,2.0,62,NaN,4,7,23,9,1,0,1,0,0,1,0,697
1,68,12.0,3,10.0,61,11.0,4,6,25,11,1,1,0,0,1,0,0,1290
2,71,12.0,3,6.0,46,12.0,2,7,21,12,1,0,1,0,0,1,0,1490
3,59,16.0,1,8.0,48,16.0,7,8,21,10,0,0,0,1,0,0,1,1720
4,48,12.0,4,6.0,39,12.0,2,9,17,13,0,1,0,0,1,0,0,1956


In [2]:
# find the correlation bwtween each variables
bwdf_corr = bwdf_low.corr(method = 'pearson')
bwdf_corr.loc[ : , 'bwght'].round(decimals = 2).sort_values(ascending = False)

bwght     1.00
omaps     0.25
fmaps     0.25
feduc     0.14
mblck     0.13
fblck     0.12
male      0.11
meduc     0.10
npvis     0.06
moth     -0.02
fwhte    -0.04
monpre   -0.05
foth     -0.08
mwhte    -0.11
fage     -0.40
mage     -0.46
cigs     -0.57
drink    -0.74
Name: bwght, dtype: float64

In [3]:
#manipulate data such as fillna missing value & convert logarithm
bwdf_low['log_bwght'] = np.log(bwdf_low['bwght'])
bwdf_low['log_mage']  = np.log(bwdf_low['mage'])
bwdf_low['log_fage']  = np.log(bwdf_low['fage'])

# filling meduc NAs with MEDIAN
meduc_median = bwdf_low['meduc'].median()
bwdf_low['meduc'].fillna(value = meduc_median,
                         inplace = True)
# filling feduc NAs with MEDIAN
feduc_median = bwdf_low['feduc'].median()
bwdf_low['feduc'].fillna(value = feduc_median,
                         inplace = True)
# filling npvis NAs with MEDIAN
npvis_median = bwdf_low['npvis'].median()
bwdf_low['npvis'].fillna(value = npvis_median,
                         inplace = True)

#checking missing value
#bwdf_low.isnull().sum(axis=0)

# counting the number of zeroes for 
cigs_zeroes   = len(bwdf_low['cigs'][bwdf_low['cigs']==0])
drink_zeroes  = len(bwdf_low['drink'][bwdf_low['drink']==0])
print(f"""
                 No\t\tYes
               ---------------------
cigarette       | {cigs_zeroes}\t\t{len(bwdf_low) - cigs_zeroes}
drink           | {drink_zeroes}\t\t{len(bwdf_low) - drink_zeroes}
""")
#Because the observations are too few it's usless to get dummies for cig and drink

# amplify the significant of two variables (cigarette & drink)
bwdf_low['cigs:drink'] = bwdf_low.loc[:,'drink']*bwdf_low.loc[:,'cigs']
bwdf_low['fage:mage'] = bwdf_low.loc[:,'fage']*bwdf_low.loc[:,'mage']
#check how many usable variables so far
bwdf_low.info()


                 No		Yes
               ---------------------
cigarette       | 9		187
drink           | 11		185

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   mage        196 non-null    int64  
 1   meduc       196 non-null    float64
 2   monpre      196 non-null    int64  
 3   npvis       196 non-null    float64
 4   fage        196 non-null    int64  
 5   feduc       196 non-null    float64
 6   omaps       196 non-null    int64  
 7   fmaps       196 non-null    int64  
 8   cigs        196 non-null    int64  
 9   drink       196 non-null    int64  
 10  male        196 non-null    int64  
 11  mwhte       196 non-null    int64  
 12  mblck       196 non-null    int64  
 13  moth        196 non-null    int64  
 14  fwhte       196 non-null    int64  
 15  fblck       196 non-null    int64  
 16  foth        196 non-null    int64  
 

Manipulate data above
#
#
#
#

In [4]:
# preparing explanatory variable data
birthweight_data   = bwdf_low.drop(['bwght','log_bwght'],
                                axis = 1)


# preparing response variable data
birthweight_target = bwdf_low.loc[ : , 'bwght']


# preparing training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
            birthweight_data,
            birthweight_target,
            test_size = 0.25,
            random_state = 219)


# checking the shapes of the datasets
print(f"""
Training Data
-------------
X-side: {x_train.shape}
y-side: {y_train.shape[0]}


Testing Data
------------
X-side: {x_test.shape}
y-side: {y_test.shape[0]}
""")


Training Data
-------------
X-side: (147, 21)
y-side: 147


Testing Data
------------
X-side: (49, 21)
y-side: 49



In [5]:
# declaring set of x-variables
x_variables = ['omaps','fmaps','cigs','drink','fage:mage']


# looping to make x-variables suitable for statsmodels
for val in x_variables:
    print(f"{val} +")

omaps +
fmaps +
cigs +
drink +
fage:mage +


In [6]:
# merging x_train and y_train so that they can be used in statsmodels
birthweight_train = pd.concat([x_train, y_train], axis = 1)

# merging x_test and y_test so that they can be used in statsmodels
birthweight_test = pd.concat([x_test, y_test], axis = 1)

#build a train model
lm_best = smf.ols(formula =  """bwght ~mage +
                                meduc +
                                monpre +
                                npvis +
                                fage +
                                feduc +
                                omaps +
                                cigs +
                                drink +
                                fmaps +
                                male +
                                mwhte +
                                mblck +
                                moth +
                                fwhte +
                                fblck +
                                foth +
                                cigs:drink +
                                fage:mage""",
                                data = birthweight_train)

#fit the train model based on the data
results = lm_best.fit()



#analyze the summary output
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.763
Model:                            OLS   Adj. R-squared:                  0.731
Method:                 Least Squares   F-statistic:                     24.39
Date:                Sun, 05 Dec 2021   Prob (F-statistic):           2.97e-32
Time:                        15:21:10   Log-Likelihood:                -1058.9
No. Observations:                 147   AIC:                             2154.
Df Residuals:                     129   BIC:                             2208.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1186.1040    470.761      2.520      0.0

In [7]:
# merging x_train and y_train so that they can be used in statsmodels
birthweight_train = pd.concat([x_train, y_train], axis = 1)

# merging x_test and y_test so that they can be used in statsmodels
birthweight_test = pd.concat([x_test, y_test], axis = 1)

#build a train model
lm_best = smf.ols(formula =  """bwght ~fmaps +
                                cigs +
                                drink +
                                fage:mage""",
                                data = birthweight_train)

#fit the train model based on the data
results = lm_best.fit()



#analyze the summary output
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.723
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                     92.51
Date:                Sun, 05 Dec 2021   Prob (F-statistic):           1.48e-38
Time:                        15:21:10   Log-Likelihood:                -1070.4
No. Observations:                 147   AIC:                             2151.
Df Residuals:                     142   BIC:                             2166.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3859.6774    449.436      8.588      0.0

In [8]:
# applying modelin scikit-learn

# preparing x-variables from the OLS model
ols_data = bwdf_low.loc[:, x_variables]


# preparing response variable
birthweight_target = bwdf_low.loc[:, 'bwght']


###############################################
## setting up more than one train-test split ##
###############################################
# FULL X-dataset (normal Y)
x_train_FULL, x_test_FULL, y_train_FULL, y_test_FULL = train_test_split(
            birthweight_data,     # x-variables
            birthweight_target,   # y-variable
            test_size = 0.25,
            random_state = 219)


# OLS p-value x-dataset (normal Y)
x_train_OLS, x_test_OLS, y_train_OLS, y_test_OLS = train_test_split(
            ols_data,         # x-variables
            birthweight_target,   # y-variable
            test_size = 0.25,
            random_state = 219)


In [9]:
# INSTANTIATING a model object
lr = LinearRegression()


# FITTING to the training data
lr_fit = lr.fit(x_train_OLS, y_train_OLS)


# PREDICTING on new data
lr_pred = lr_fit.predict(x_test_OLS)


# SCORING the results
print('OLS Training Score :', lr.score(x_train_OLS, y_train_OLS).round(4))  # using R-square
print('OLS Testing Score  :',  lr.score(x_test_OLS, y_test_OLS).round(4)) # using R-square

lr_train_score = lr.score(x_train_OLS, y_train_OLS).round(4)
lr_test_score  = lr.score(x_test_OLS, y_test_OLS).round(4)

# displaying and saving the gap between training and testing
print('OLS Train-Test Gap :', abs(lr_train_score - lr_test_score).round(4))
lr_test_gap = abs(lr_train_score - lr_test_score).round(4)

OLS Training Score : 0.7231
OLS Testing Score  : 0.6783
OLS Train-Test Gap : 0.0448


In [10]:
# zipping each feature name to its coefficient
lr_model_values = zip(birthweight_data[x_variables].columns,
                      lr_fit.coef_.round(decimals = 2))


# setting up a placeholder list to store model features
lr_model_lst = [('intercept', lr_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in lr_model_values:
    lr_model_lst.append(val)
    

# checking the results
for pair in lr_model_lst:
    print(pair)

('intercept', 3905.14)
('omaps', 10.48)
('fmaps', 78.02)
('cigs', -37.05)
('drink', -105.4)
('fage:mage', -0.22)


In [11]:
# INSTANTIATING a model object
lasso_model = sklearn.linear_model.Lasso(alpha = 1.0,
                                         normalize = True) # default magitude


# FITTING to the training data
lasso_fit = lasso_model.fit(x_train_OLS, y_train_OLS)


# PREDICTING on new data
lasso_pred = lasso_fit.predict(x_test_OLS)


# SCORING the results
print('Lasso Training Score :', lasso_model.score(x_train_OLS, y_train_OLS).round(4))
print('Lasso Testing Score  :', lasso_model.score(x_test_OLS, y_test_OLS).round(4))


## the following code has been provided for you ##

# saving scoring data for future use
lasso_train_score = lasso_model.score(x_train_OLS, y_train_OLS).round(4) # using R-square
lasso_test_score  = lasso_model.score(x_test_OLS, y_test_OLS).round(4)   # using R-square


# displaying and saving the gap between training and testing
print('Lasso Train-Test Gap :', abs(lasso_train_score - lasso_test_score).round(4))
lasso_test_gap = abs(lasso_train_score - lasso_test_score).round(4)

Lasso Training Score : 0.7223
Lasso Testing Score  : 0.6841
Lasso Train-Test Gap : 0.0382


C:\Users\yclee\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


In [12]:
# zipping each feature name to its coefficient
lasso_model_values = zip(birthweight_data.columns, lasso_fit.coef_.round(decimals = 2))


# setting up a placeholder list to store model features
lasso_model_lst = [('intercept', lasso_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in lasso_model_values:
    lasso_model_lst.append(val)
    

# checking the results
for pair in lasso_model_lst:
    print(pair)

('intercept', 3973.34)
('mage', 7.35)
('meduc', 69.4)
('monpre', -35.76)
('npvis', -103.82)
('fage', -0.22)


In [13]:
# INSTANTIATING a model object
ard_model = sklearn.linear_model.ARDRegression(normalize = False)


# FITTING the training data
ard_fit = ard_model.fit(x_train_OLS, y_train_OLS)


# PREDICTING on new data
ard_pred = ard_fit.predict(x_test_OLS)


print('Training Score:', ard_model.score(x_train_OLS, y_train_OLS).round(4))
print('Testing Score :', ard_model.score(x_test_OLS, y_test_OLS).round(4))


# saving scoring data for future use
ard_train_score = ard_model.score(x_train_OLS, y_train_OLS).round(4)
ard_test_score  = ard_model.score(x_test_OLS, y_test_OLS).round(4)


# displaying and saving the gap between training and testing
print('ARD Train-Test Gap :', abs(ard_train_score - ard_test_score).round(4))
ard_test_gap = abs(ard_train_score - ard_test_score).round(4)

Training Score: 0.7222
Testing Score : 0.6849
ARD Train-Test Gap : 0.0373


C:\Users\yclee\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


In [14]:
# zipping each feature name to its coefficient
ard_model_values = zip(birthweight_data.columns, ard_fit.coef_.round(decimals = 5))


# setting up a placeholder list to store model features
ard_model_lst = [('intercept', ard_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in ard_model_values:
    ard_model_lst.append(val)
    

# checking the results
for pair in ard_model_lst:
    print(pair)

('intercept', 4038.07)
('mage', 0.00177)
('meduc', 72.83036)
('monpre', -36.67285)
('npvis', -106.22424)
('fage', -0.22304)


In [15]:
# comparing results

print(f"""
Model      Train Score      Test Score
-----      -----------      ----------
OLS        {lr_train_score}           {lr_test_score}
Lasso      {lasso_train_score}           {lasso_test_score}
*ARD       {ard_train_score}           {ard_test_score}

* final model
""")


# creating a dictionary for model results
model_performance = {
    
    'Model Type'    : ['OLS', 'Lasso', 'ARD'],
           
    'Training' : [lr_train_score, lasso_train_score,
                                   ard_train_score],
           
    'Testing'  : [lr_test_score, lasso_test_score,
                                   ard_test_score],
                    
    'Train-Test Gap' : [lr_test_gap, lasso_test_gap,
                                        ard_test_gap],
                    
    'Model Size' : [len(lr_model_lst), len(lasso_model_lst),
                                    len(ard_model_lst)],
                    
    'Model' : [lr_model_lst, lasso_model_lst, ard_model_lst]}


# converting model_performance into a DataFrame
#model_performance = pd.DataFrame(model_performance)


# sending model results to Excel
#model_performance.to_excel('./__model_results/linear_model_performance.xlsx',
                           #index = False)


Model      Train Score      Test Score
-----      -----------      ----------
OLS        0.7231           0.6783
Lasso      0.7223           0.6841
*ARD       0.7222           0.6849

* final model



In [16]:
print(f"""\n\n\n {'*' * 95}""")

#final model output
print(f"""
\tIn this exercise I using three model which is OLS, Lasso, and ARD to run my regression,\n 
although they only have minute difference\033[1m"{(lr_train_score-lasso_train_score).round(decimals=4)}"\033[0m. The Train-Test Gap has a significant change,\n
in this case smaller is better therefore I decided to take\033[1m "ARD" \033[0mas my final model.\n
The \033[1mTraining Score\033[0m is {ard_train_score} and the \033[1mTesting Score\033[0m is {ard_test_score} which make my \033[1mTrain-Test Gap\033[0m around\n
{ard_test_gap.round(decimals=3)}, that give me a great precision of prediction model.""")
print(f"""\n\n\n {'*' * 95}""")




 ***********************************************************************************************

	In this exercise I using three model which is OLS, Lasso, and ARD to run my regression,
 
although they only have minute difference"0.0008". The Train-Test Gap has a significant change,

in this case smaller is better therefore I decided to take "ARD" as my final model.

The Training Score is 0.7222 and the Testing Score is 0.6849 which make my Train-Test Gap around

0.037, that give me a great precision of prediction model.



 ***********************************************************************************************
